In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip install statsmodels

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels as sm

In [ ]:
df_oil = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv")
df_holiday = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv")
df_stores = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
df_train = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv", parse_dates=True)
df_test = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv")
df_transactions = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv")

In [ ]:
#df_stores.head()
#df_stores.describe()
df_train.head(100)
#df_oil.head()

In [ ]:
df_holiday.head()

In [ ]:
print("Columns of df_train:", df_train.columns)
print("distinct count of store nbr", len(df_train.store_nbr.unique()))
print("distinct count of family", len(df_train.family.unique()))
print("Start date", min(df_train.date), "end date", max(df_train.date))
df_train

In [ ]:
def get_fresh_dataset():
    df_oil = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv")
    df_holiday = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv")
    df_stores = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
    df_train = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv", parse_dates=True)
    df_test = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv")
    df_transactions = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv")
 
    df_train = df_train.join(df_stores, on='store_nbr', rsuffix='_store')
    df_train = df_train.join(df_transactions, on='store_nbr', rsuffix='_transaction')
    df_oil.set_index('date', inplace=True)
    df_holiday.set_index('date', inplace=True)
    df_train = df_train.join(df_oil, on='date')
    df_train = df_train.join(df_holiday, on='date', rsuffix='_holiday')
    return df_train

In [ ]:
df_train=get_fresh_dataset()

Let's go column by column and try to find interesting things. Then I'll try to find patterns by comparing columns. 

Let's start with store_nbr

In [ ]:
# Calculating % share of each store in the dataset

#df_train.groupby("store_nbr").agg("count").reset_index() * 100/ len(df_train)
# So the dataset is balanced with respect to store_nbr
df_train1=df_train
#df_train.groupby("store_nbr").agg("count").reset_index() * 100/ len(df_train)
df_train1=df_train1.groupby("store_nbr").agg("count").reset_index() * 100/ len(df_train)
df_train1=df_train1.rename(columns={"store_nbr":"store_share"})
#df_train.head(10) 
#df_train2=df_train1['store_nbr']
#df_train.loc[store_nbr]
#df_train1
df_train1.drop(['id','date','family','sales','onpromotion'],axis=1,inplace=True)
#id	date	store_nbr	family	sales	onpromotion
df_train1

In [ ]:
# Calculating % sales share of each store in the dataset
#df_train
#df_train.groupby("store_nbr").agg("sum").reset_index().sort_values(by='sales', ascending=False) * 100 / sum(df_train.sales)
# So store 43 has maximum share of 5.78% and store 51 has minimum share of 0.25%
#df_train.groupby("store_nbr").count().reset_index().sort_values(by='sales', ascending=False) * 100 / sum(df_train.sales)

#Feature2 = df_train.groupby(["family", "store_nbr"]).tail(60).groupby(["family", "store_nbr"]).sales.sum().reset_index()c[c.sales == 0]
#df_train1.drop(['id','date','family','sales','onpromotion'],axis=1,inplace=True)
df_train2=df_train
df_train2=df_train2.groupby("store_nbr").agg("sum").reset_index().sort_values(by='sales', ascending=False) * 100 / sum(df_train.sales)
df_train2=df_train2.rename(columns={"onpromotion":"store_sell_share"})
df_train2.drop(['store_nbr','id','sales'],axis=1,inplace=True)
df_train2

In [ ]:
# Promotions per store

df_train.groupby("store_nbr")['onpromotion'].apply(lambda x: (x > 0).sum()).reset_index().sort_values(by='onpromotion', ascending=False)
# So store 2 has highest number of items promoted. Store 51 had lowest. 

In [ ]:
# Removing outliers: 
# 1. Removing outliers per store
# 2. Removing outliers from whole dataset
# for i in range(1, 55):
#     pass
#     mean = mean(df_train[df_train.store_nbr==i].sales)
#     std = np.std(df_train[df_train.store_nbr==i].sales)
#     df_train = df_train[df_train.store_nbr==i]


mean = np.mean(df_train.sales)
std = np.std(df_train.sales)
range_min = mean - 3 * std
range_max = mean + 3 * std
df_train = df_train[(df_train.sales<range_max) & (df_train.sales>range_min)]


In [ ]:
# Average sales per store_nbr per year
df_train['year'] = pd.DatetimeIndex(df_train.date).year
df_train['month'] = pd.DatetimeIndex(df_train.date).month

df_mean_sales = df_train.groupby(['store_nbr', 'year']).mean().reset_index()
df_mean_sales_overall = df_train.groupby(['year']).mean().reset_index()
df_mean_sales_monthly_overall = df_train.groupby(['year', 'month']).mean().reset_index()

In [ ]:
for i in range(11, 20):
    condition = (df_mean_sales.store_nbr == i)
    plt.plot(df_mean_sales[condition]['year'], df_mean_sales[condition]['sales'])

In [ ]:
plt.plot(df_mean_sales_overall['year'], df_mean_sales_overall['sales'])

Looks like the average sales has been increasing year on year. 
Let's plot monthly average. 

In [ ]:
df_mean_sales_monthly_overall['year_month'] = df_mean_sales_monthly_overall.apply(lambda x: str(x['year']) + '_' + str(x['month']), axis=1)
plt.plot(df_mean_sales_monthly_overall['year_month'], df_mean_sales_monthly_overall['sales'])

Again increasing trend. 

De-trending it. 

In [ ]:
df_train['detrend_sales'] = df_train.sales.diff()


In [ ]:
df_train[['sales', 'detrend_sales']]
# dt suffix stands for detrend
df_mean_sales_dt = df_train.groupby(['store_nbr', 'year']).mean().reset_index()
df_mean_sales_overall_dt = df_train.groupby(['year']).mean().reset_index()
df_mean_sales_monthly_overall_dt = df_train.groupby(['year', 'month']).mean().reset_index()

In [ ]:
df_mean_sales_monthly_overall_dt['year_month'] = df_mean_sales_monthly_overall_dt.apply(lambda x: str(x['year']) + '_' + str(x['month']), axis=1)
plt.plot(df_mean_sales_monthly_overall_dt['year_month'], df_mean_sales_monthly_overall_dt['detrend_sales'])

In [ ]:
max_data=df_train['sales'].max()
min_data=df_train['sales'].min()
total_count=df_train['sales'].count()
bin1 = (max_data-min_data) / total_count
bin1

In [ ]:
df_train['sales'].plot(kind='hist',bin,=0.041,figsize=(10,7), vert=True,color='navy')
plt.ylabel('percentage of share of store')
plt.show()

In [ ]:
# Plotting time vs sales for store 2(random)

df_temp = df_train[(df_train.store_nbr == 2)]
df_temp = df_temp[df_temp.date<='2013-02-01']
plt.plot(df_temp['date'], df_temp['sales'])

# df_temp = df_train[df_train.store_nbr == 3]
# df_temp = df_temp[df_temp.date<='2018-01-01']
# plt.plot(df_temp['date'], df_temp['sales'])


Models to try: 
1. Croston
2. Croston TSB
3. Exponential smoothing
4. ARIMA

Remove trend and use moving average then use the following models: 
1. Linear regression(Stats model)
2. Quantile regression
3. Deep learning(tensorflow)

In [ ]:
df_train.head()

In [ ]:
# Joining all tables

df_oil = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv")
df_holiday = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv")
df_stores = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
df_train = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv", parse_dates=True)
df_test = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv")
df_transactions = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv")

df_train = df_train.join(df_stores, on='store_nbr', rsuffix='_store')
df_train = df_train.join(df_transactions, on='store_nbr', rsuffix='_transaction')
df_oil.set_index('date', inplace=True)
df_holiday.set_index('date', inplace=True)
df_train = df_train.join(df_oil, on='date')
df_train = df_train.join(df_holiday, on='date', rsuffix='_holiday')



In [ ]:
df_train.set_index('id', inplace=True)

In [ ]:
df_train.reset_index(inplace==True)

In [ ]:
# Average sales trend per family per month

df_train.date = pd.DatetimeIndex(df_train['date'])
df_train['year'] = df_train.date.dt.year
df_train['month'] = df_train.date.dt.month
df_train['day'] = df_train.date.dt.day


In [ ]:
# Average sales trend per month by family, city, state, locale, and store_nbr for top 5 by sales

df_family = df_train.groupby(['family', 'year', 'month']).agg('mean').reset_index()[['family', 'year', 'month', 'sales']]
df_city = df_train.groupby(['city', 'year', 'month']).agg('mean').reset_index()[['city', 'year', 'month', 'sales']]
df_locale = df_train.groupby(['locale', 'year', 'month']).agg('mean').reset_index()[['locale', 'year', 'month', 'sales']]
df_state = df_train.groupby(['state', 'year', 'month']).agg('mean').reset_index()[['state', 'year', 'month', 'sales']]
df_store_nbr = df_train.groupby(['store_nbr', 'year', 'month']).agg('mean').reset_index()[['store_nbr', 'year', 'month', 'sales']]

df_family['day'] = 1
df_city['day'] = 1
df_locale['day'] = 1
df_state['day'] = 1
df_store_nbr['day'] = 1

# df_family['year_month'] = df_family['month'].astype('str') + '_' + df_family['year'].astype('str')
# df_city['year_month'] = df_city['month'].astype('str') + '_' + df_city['year'].astype('str')
# df_locale['year_month'] = df_locale['month'].astype('str') + '_' + df_locale['year'].astype('str')
# df_state['year_month'] = df_state['month'].astype('str') + '_' + df_state['year'].astype('str')
# df_store_nbr['year_month'] = df_store_nbr['month'].astype('str') + '_' + df_store_nbr['year'].astype('str')


df_family['year_month'] = pd.to_datetime(df_family[['year', 'month', 'day']])
df_city['year_month'] = pd.to_datetime(df_city[['year', 'month', 'day']])
df_locale['year_month'] = pd.to_datetime(df_locale[['year', 'month', 'day']])
df_state['year_month'] = pd.to_datetime(df_state[['year', 'month', 'day']])
df_store_nbr['year_month'] = pd.to_datetime(df_store_nbr[['year', 'month', 'day']])


In [ ]:
df_family['year_month']

In [ ]:
top_5_family_by_sales = list(df_family.groupby('family').agg('sum').reset_index().sort_values(by='sales').head(5)['family'])
top_5_city_by_sales = list(df_city.groupby('city').agg('sum').reset_index().sort_values(by='sales').head(5)['city'])
top_5_locale_by_sales = list(df_locale.groupby('locale').agg('sum').reset_index().sort_values(by='sales').head(5)['locale'])
top_5_state_by_sales = list(df_state.groupby('state').agg('sum').reset_index().sort_values(by='sales').head(5)['state'])

In [ ]:
df_family = df_family.pivot_table(index='year_month', columns='family', values='sales')[top_5_family_by_sales]
df_city = df_city.pivot_table(index='year_month', columns='city', values='sales')[top_5_city_by_sales]
df_locale = df_locale.pivot_table(index='year_month', columns='locale', values='sales')[top_5_locale_by_sales]
df_state = df_state.pivot_table(index='year_month', columns='state', values='sales')[top_5_state_by_sales]

In [ ]:
fig = plt.figure()

family_subplot=fig.add_subplot(3, 2, 1)
city_subplot = fig.add_subplot(3, 2, 2)
locale_subplot=fig.add_subplot(3, 2, 3)
state_subplot = fig.add_subplot(3, 2, 4)
# store_nbr_subplot = fig.add_subplot(3, 2)

figure_size = (25, 25)

xticks = df_family.index

df_family.plot(kind='line', figsize=figure_size, ax=family_subplot)
family_subplot.set_title("Sales trend by family")
family_subplot.set_ylabel("Average sales")
family_subplot.set_xlabel("Years")
# family_subplot.set_xticks(df_family.index)
# family_subplot.set_xticklabels(df_family.index, rotation=90)

df_city.plot(kind='line', figsize=figure_size, ax=city_subplot)
city_subplot.set_title("Sales trend by city")
city_subplot.set_ylabel("Average sales")
city_subplot.set_xlabel("Years")

df_locale.plot(kind='line', figsize=figure_size, ax=locale_subplot)
locale_subplot.set_title("Sales trend by locale")
locale_subplot.set_ylabel("Average sales")
locale_subplot.set_xlabel("Years")

df_state.plot(kind='line', figsize=figure_size, ax=state_subplot)
state_subplot.set_title("Sales trend by state")
state_subplot.set_ylabel("Average sales")
state_subplot.set_xlabel("Years")

## Let's plot average sales per month with trend line.

In [ ]:
df_family

In [ ]:
df_trend = df_train.groupby(['year', 'month']).mean().reset_index()[['year', 'month', 'sales']]
df_trend['day'] = 1 # First of every month

df_trend_temp = df_trend

df_trend['date'] = pd.to_datetime(df_trend[['year', 'month', 'day']])

df_trend = df_trend[['date', 'sales']]

df_trend.set_index('date', inplace=True)
df_trend.plot(kind='line')

z = np.polyfit(df_trend_temp[['year', 'month', 'day']], df_trend_temp['sales'], 1)
p = np.poly1d(z)
plt.plot(df_trend_temp[['year', 'month', 'day']],p(df_trend.index),"r--")

plt.show()


In [ ]:
# Percentage zeros in sales
len(df_train[df_train.sales==0]) / len(df_train)


In [ ]:
# Croston

def croston(ts, periods=1, alpha=0.4):
    """
    """
    d = np.array(ts)
    col = len(d)
    d = np.append(d, [np.nan]*extra_periods)
    a, p, f = np.full(3, cols+extra_periods, np.nan)
    q = 1
    

In [ ]:
#question 2
#Explore your features further in their distributions and plot their box plots. Show outliers 
#for each feature. Do you think any of the outliers may impact your analysis? Why? Provide 
#supporting visualizations with their analysis

df_train1['store_share'].plot(kind='box',figsize=(10,7), vert=True, color='navy')
plt.ylabel('percentage of share of store')
plt.show()


In [ ]:
df_train2['store_sell_share'].plot(kind='box',figsize=(10,7), vert=True, color='navy')
plt.ylabel('percentage of sell share of store')
plt.show()


In [ ]:
df_transactions1['transactions'].plot(kind='box',figsize=(10,7), vert=True, color='navy')
plt.ylabel('percentage of sell share of store')
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
df_transactions["date"] = pd.to_datetime(df_transactions.date)
a = df_transactions
a["year"] = a.date.dt.year
a["month"] = a.date.dt.month
px.box(a, x="year", y="transactions" , color = "month", title = "Transactions")

In [ ]:
import pandas as pd
import numpy as np
df_oil["date"] = pd.to_datetime(df_oil.date)
# Resample
df_oil = df_oil.set_index("date").dcoilwtico.resample("D").sum().reset_index()
# Interpolate
df_oil["dcoilwtico"] = np.where(df_oil["dcoilwtico"] == 0, np.nan, df_oil["dcoilwtico"])
df_oil["dcoilwtico_interpolated"] =df_oil.dcoilwtico.interpolate()
# Plot
p = df_oil.melt(id_vars=['date']+list(df_oil.keys()[5:]), var_name='Legend')
px.line(p.sort_values(["Legend", "date"], ascending = [False, True]), x='date', y='value', color='Legend',title = "Daily Oil Price" )

In [ ]:
df_oil['dcoilwtico'].plot(kind='box',figsize=(10,7), vert=True, color='navy')
plt.ylabel('Oil price')
plt.show()

In [ ]:
df_oil